In [ ]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))



In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy
from loguru import logger
import dataloader 
import numpy as np

In [ ]:
import importlib

import tf_models  # First, import the library
importlib.reload(tf_models)  # Now, reload it


In [ ]:
import dataloader

dataset = "cifar10"

loader = getattr(dataloader, f"load_{dataset}")
x_train, y_train, x_test, y_test = loader(onehot=True)

logger.info(x_train.shape)
logger.info(x_test.shape)
if x_train[0].ndim == 2:
    x_train = x_train[..., np.newaxis]
    x_test = x_test[..., np.newaxis]
image_shape = x_train[0].shape
logger.info(f"image shape: {image_shape}")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.losses import CategoricalCrossentropy
import matplotlib.pyplot as plt
import numpy as np
import dataloader

from tf_models import *

model = AlexNet(image_shape, num_classes=y_train.shape[1], augmentation=True, l2_lambda=0.0002)




In [ ]:
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
#optimizer = 'adam'
# 0.1 lr = ~12%
# 0.01 lr = 0.2188

model.compile(
    optimizer=optimizer, loss=CategoricalCrossentropy(), metrics=["accuracy"]
)

with tf.device("/GPU:0"):
    history = model.fit(
        x_train, y_train, epochs=7, batch_size=64, validation_split=0.2
    )
# Evaluate the model on the test set
# loss, accuracy = model.evaluate(x_test, y_test)
# print(f"Test loss: {loss:.4f}")
# print(f"Test accuracy: {accuracy:.4f}")

In [ ]:
model.save('viz.keras')

In [ ]:
#model.load_weights('viz.keras')

In [ ]:
# find the right index for category X
from collections import defaultdict
import random

m = defaultdict(list)
for i, y in enumerate(y_train):
    y = np.argmax(y)
    m[y].append(i)
    
for i in range(10):
    random.shuffle(m[i])
    print(i, m[i][:3])

In [ ]:
# CNN viz
import math
from viz import vizn_1d, vizn_3d


img_index = [m[5][0],m[5][1],m[8][0],m[8][1]]
n = len(img_index)

img = x_train[img_index]
label = y_train[img_index]
input = x_train[img_index]

fig, imgxs = plt.subplots(1, n, figsize=(10, 5))
print(imgxs)
for i in range(n):
    imgxs[i].imshow(img[i], cmap='viridis')
plt.show()

ls = model.layers
os = []
o = input
for l in ls:
    o = l(o, training=False)
    os.append(o)
    logger.info(o.shape)
    logger.info(l.name)
    if o.ndim == 2:
        vizn_1d(o)
    if o.ndim == 4:
        vizn_3d(o)
        #vizn_3d(o[:,:,:,0:1])
        #logger.info(o[0:1,:,:,1:2])
        #logger.info(l.weights)
        pass
#logger.info(o)

logger.info([np.argmax(la) for la in label])
logger.info([np.argmax(oi) for oi in o])